GPU Parallel Huffman Encoding/Decoding Project

This Jupyter Notebook uses Numba and CuPy, the recommended tools for GPU computing in the RAPIDS environment.

1. Setup and Utility Functions

==============================================================================

In [ ]:
import numpy as np
import cupy as cp
from numba import cuda
import time
import math
import matplotlib.pyplot as plt
from collections import Counter
import heapq
import os

Configuration (Adjust these based on your tests)

In [ ]:
THREAD_PER_BLOCK = 256
MAX_SYMBOLS = 256 # For byte-level compression
MAX_CODE_LENGTH = 32 # Maximum bits for a Huffman code

--- Utility Functions ---

In [ ]:
def generate_test_data(size_mb, redundancy_level='medium'):
"""Generates a byte array of a specific size with varying redundancy."""
size_bytes = size_mb * 1024 * 1024

if redundancy_level == 'low':
    # Near-uniform distribution (high entropy)
    return np.random.randint(0, MAX_SYMBOLS, size=size_bytes, dtype=np.uint8)
elif redundancy_level == 'high':
    # Highly skewed distribution (low entropy)
    popular_symbols = np.random.choice(np.arange(MAX_SYMBOLS), size=int(MAX_SYMBOLS * 0.1), replace=False)
    weights = np.zeros(MAX_SYMBOLS)
    weights[popular_symbols] = np.random.rand(len(popular_symbols))
    weights /= weights.sum()
    return np.random.choice(np.arange(MAX_SYMBOLS), size=size_bytes, p=weights, replace=True).astype(np.uint8)
else: # medium (standard text-like distribution)
    # 10 characters are frequent, others are less frequent
    weights = np.array([0.05] * 10 + [0.0038] * (MAX_SYMBOLS - 10))
    weights[0] += 1.0 - weights.sum() # Normalize
    weights = np.maximum(weights, 0.0001) # Ensure no zero weights
    weights /= weights.sum()
    return np.random.choice(np.arange(MAX_SYMBOLS), size=size_bytes, p=weights, replace=True).astype(np.uint8)

2. Sequential Baseline Implementation (CPU)
==============================================================================

In [ ]:
def cpu_huffman_encode(data):
"""
Standard sequential Huffman encoding (all stages combined) on CPU.
Returns: code_table, compressed_bits, original_size
"""
# 1. Frequency Counting (Sequential)
freq = Counter(data)

# 2. Code Assignment (Sequential Tree Construction)
if not freq:
    return {}, b'', 0

# Build the Huffman Tree
heap = [[weight, [symbol, ""]] for symbol, weight in freq.items()]
heapq.heapify(heap)

while len(heap) > 1:
    lo = heapq.heappop(heap)
    hi = heapq.heappop(heap)
    for pair in lo[1:]:
        pair[1] = '0' + pair[1]
    for pair in hi[1:]:
        pair[1] = '1' + pair[1]
    heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    
code_table = dict(lo[1:][i] for i in range(len(lo[1:])))

# 3. Bitstream Encoding (Sequential)
encoded_bits = ""
for symbol in data:
    encoded_bits += code_table[symbol]
    
# Pack bits into bytes
padding_bits = 8 - (len(encoded_bits) % 8)
encoded_bits += '0' * padding_bits

# Header: Store padding info and code table
header = str(padding_bits) + '|' + str(code_table) + '|'

compressed_bytes = bytearray()
for i in range(0, len(encoded_bits), 8):
    byte = encoded_bits[i:i+8]
    compressed_bytes.append(int(byte, 2))
    
return header.encode('utf-8') + bytes(compressed_bytes), len(data)

3. Parallel Implementation (GPU Kernels)

==============================================================================

--- Stage 1: Parallel Frequency Counting (CUDA Kernel) ---

In [ ]:
@cuda.jit
def parallel_frequency_count(data, freq_out):
"""
Parallel Frequency Counting using a global atomic operation.
Each thread processes one byte and increments the corresponding frequency count.
"""
idx = cuda.grid(1)
if idx < data.size:
# Use atomic add to safely increment the frequency count for the symbol
# data[idx] is the symbol (byte value 0-255)
# freq_out is the 256-element array storing counts
cuda.atomic.add(freq_out, data[idx], 1)

def gpu_frequency_stage(data_gpu, freq_out_gpu):
"""Wrapper to launch the frequency counting kernel."""
blocks_per_grid = (data_gpu.size + THREAD_PER_BLOCK - 1) // THREAD_PER_BLOCK
parallel_frequency_count[blocks_per_grid, THREAD_PER_BLOCK](data_gpu, freq_out_gpu)
cuda.synchronize()

--- Stage 2: Parallel Canonical Code Assignment (CPU/GPU Hybrid) ---

Canonical Huffman is used because tree traversal (non-canonical) is highly sequential.

def canonical_code_assignment(freq_gpu):
"""
Hybrid stage: CPU calculates lengths (sequential), then GPU generates codes (parallel).
"""
freq_cpu = cp.asnumpy(freq_gpu)

# 1. Sequential Code Length Determination (Huffman Algorithm)
active_freq = {i: f for i, f in enumerate(freq_cpu) if f > 0}
if not active_freq:
    return cp.array([0]*MAX_SYMBOLS, dtype=cp.uint8), {}

# Initial list of nodes: (frequency, symbol)
heap = [[f, s] for s, f in active_freq.items()]
heapq.heapify(heap)

# Dictionary to store (symbol -> depth)
depths = {s: 0 for s in active_freq.keys()}

# Build the tree and record depths
while len(heap) > 1:
    lo_f, lo_s = heapq.heappop(heap)
    hi_f, hi_s = heapq.heappop(heap)
    
    # Merge operation
    new_s = (lo_s, hi_s) # Internal node represented by a tuple of children
    new_f = lo_f + hi_f
    heapq.heappush(heap, [new_f, new_s])
    
    # Increment depth of all children
    for symbol_or_tuple in [lo_s, hi_s]:
        if isinstance(symbol_or_tuple, int): # Leaf node (actual symbol)
            depths[symbol_or_tuple] += 1
        else: # Internal node (a tuple of children)
            q = [symbol_or_tuple]
            while q:
                current = q.pop(0)
                if isinstance(current, int):
                    depths[current] += 1
                else:
                    q.extend(list(current))

# 2. Parallel Canonical Code Generation (GPU)
# The depth (length) is the only thing needed.
code_lengths_cpu = np.zeros(MAX_SYMBOLS, dtype=np.uint8)
for s, d in depths.items():
    code_lengths_cpu[s] = d

code_lengths_gpu = cp.asarray(code_lengths_cpu)

# Sort symbols by length, then by symbol value (canonical order)
# We will let the CPU handle the sorting of lengths and symbols, then generate codes on the GPU if complexity required it,
# but since the generation itself is sequential once lengths are known, we'll keep the construction on the CPU for simplicity.

# Final code generation (CPU-side for simplicity, but easily done on GPU)
codes = {}

# Get sorted symbols and lengths: [(length, symbol), ...]
sorted_pairs = sorted([(code_lengths_cpu[i], i) for i in range(MAX_SYMBOLS) if code_lengths_cpu[i] > 0])

if not sorted_pairs:
    return code_lengths_gpu, {}

current_code = 0
current_length = 0

for length, symbol in sorted_pairs:
    if length > current_length:
        if current_length > 0:
            current_code = (current_code + 1) << (length - current_length)
        current_length = length
    
    codes[symbol] = format(current_code, f'0{length}b')
    current_code += 1

return code_lengths_gpu, codes

--- Stage 3: Parallel Bitstream Encoding (CUDA Kernel & CuPy Scan) ---

In [ ]:
@cuda.jit
def parallel_write_bitstream(data_in, codes_gpu, code_lengths_gpu, start_bit_pos, output_bits, bitstream_size):
"""
Final CUDA kernel to write the packed bitstream.
Each thread writes one full code at its pre-calculated start position.
This kernel is complex due to bit-level memory addressing and potential cross-byte writes.
"""
idx = cuda.grid(1)

if idx < data_in.size:
    symbol = data_in[idx]
    code_length = code_lengths_gpu[symbol]
    
    if code_length > 0:
        # Code is stored as a 64-bit integer (max length 32 bits, but using 64 for safety)
        # codes_gpu[symbol] contains the integer representation of the code
        code_value = codes_gpu[symbol]
        start_bit = start_bit_pos[idx]
        
        # Write the code bit by bit (simplified and safe approach)
        for i in range(code_length):
            # Calculate the position in the final bitstream array
            current_bit_index = start_bit + i
            byte_index = current_bit_index // 8
            bit_offset = 7 - (current_bit_index % 8) # Write from MSB (bit 7) to LSB (bit 0)

            # Check bounds safety (avoid writing beyond allocated output size)
            if byte_index < bitstream_size:
                # Extract the i-th bit from the code_value (reading from right to left, LSB to MSB)
                bit_to_write = (code_value >> (code_length - 1 - i)) & 1
                
                # Atomic operation to safely set the bit in the output byte
                if bit_to_write == 1:
                    mask = 1 << bit_offset
                    cuda.atomic.or_(output_bits, byte_index, mask)


def gpu_encoding_stage(data_gpu, code_lengths_gpu, codes_dict):
"""Wrapper for parallel encoding using CuPy for scan and Numba for write."""
N = data_gpu.size

# Convert code dictionary to GPU arrays for lookup
code_values_cpu = np.zeros(MAX_SYMBOLS, dtype=np.uint64)
code_lengths_cpu = cp.asnumpy(code_lengths_gpu)

for symbol, code_str in codes_dict.items():
    # Store code value as an integer
    code_values_cpu[symbol] = int(code_str, 2)

codes_gpu = cp.asarray(code_values_cpu)

# 1. Parallel Code Length Lookup
# Get the length of each symbol's code for every symbol in the input data
code_lengths_in_data = code_lengths_gpu[data_gpu]

# 2. Parallel Prefix Sum (Scan)
# Compute the starting bit position for each code using an exclusive scan
# This is highly efficient in CuPy (and RAPIDS)
start_bit_pos = cp.cuda.runtime.getDevice() # Ensure the scan runs on the selected device
start_bit_pos = cp.cumsum(code_lengths_in_data, dtype=cp.uint64, axis=0)

# Shift result for exclusive scan (starting index of the *current* code)
start_bit_pos = cp.roll(start_bit_pos, 1)
start_bit_pos[0] = 0

total_bits = cp.sum(code_lengths_in_data).item()
total_bytes = math.ceil(total_bits / 8)

# 3. Parallel Bitstream Write
output_bits = cp.zeros(total_bytes, dtype=cp.uint8)

blocks_per_grid = (N + THREAD_PER_BLOCK - 1) // THREAD_PER_BLOCK

# Launch the kernel to write the codes in parallel
parallel_write_bitstream[blocks_per_grid, THREAD_PER_BLOCK](
    data_gpu, codes_gpu, code_lengths_gpu, start_bit_pos, output_bits, total_bytes
)
cuda.synchronize()

# Pad info for header
padding_bits = (8 - (total_bits % 8)) % 8

return output_bits, total_bits, padding_bits

4. Multi-GPU / Multi-Core Parallelization Structure

==============================================================================

In [ ]:
class ParallelHuffman:
def init(self, gpu_ids=[0]):
self.gpu_ids = gpu_ids
self.num_gpus = len(gpu_ids)
self.devices = [cp.cuda.Device(i) for i in gpu_ids]

def run_multi_gpu_encoding(self, data_cpu):
    """
    Simulates Multi-GPU encoding by distributing the file chunks.
    The overall parallelization is based on dividing the input data.
    """
    N = data_cpu.size
    chunk_size = math.ceil(N / self.num_gpus)
    
    chunk_data = []
    for i in range(self.num_gpus):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, N)
        chunk_data.append(data_cpu[start:end])
    
    start_time = time.time()
    
    # --- Stage 1: Parallel Frequency Counting Across GPUs ---
    total_freq_cpu = np.zeros(MAX_SYMBOLS, dtype=np.int32)
    
    for i, device in enumerate(self.devices):
        with device:
            # 1a. Copy chunk data to GPU i
            data_gpu = cp.asarray(chunk_data[i])
            freq_out_gpu = cp.zeros(MAX_SYMBOLS, dtype=cp.int32)
            
            # 1b. Run frequency counting kernel on GPU i
            gpu_frequency_stage(data_gpu, freq_out_gpu)
            
            # 1c. Collect results (Atomic Merge in Global Memory is better, but here we merge on CPU/Host)
            total_freq_cpu += cp.asnumpy(freq_out_gpu)
    
    # --- Stage 2: Code Assignment (Centralized on CPU) ---
    # The tree construction (Code Assignment) is inherently centralized/sequential 
    # based on the total frequency map. It must be computed once on the host.
    total_freq_gpu = cp.asarray(total_freq_cpu)
    code_lengths_gpu_master, codes_dict_master = canonical_code_assignment(total_freq_gpu)
    
    # --- Stage 3: Parallel Bitstream Encoding Across GPUs ---
    compressed_chunks = []
    total_bits = 0
    
    for i, device in enumerate(self.devices):
        with device:
            data_gpu = cp.asarray(chunk_data[i])
            
            # Ensure the master codes are available on the device
            code_lengths_gpu = code_lengths_gpu_master.copy()
            
            # Run encoding stage on GPU i
            chunk_bits, chunk_total_bits, chunk_padding = gpu_encoding_stage(
                data_gpu, code_lengths_gpu, codes_dict_master
            )
            
            # Collect chunk results
            compressed_chunks.append(cp.asnumpy(chunk_bits))
            total_bits += chunk_total_bits
    
    end_time = time.time()
    
    # Final Concatenation and Header
    # The true challenge is the *bit-level* concatenation across chunks.
    # Since we cannot perfectly bit-pack the last bits of chunk(i) with the first bits of chunk(i+1) in this simulation,
    # we will use the most common parallel strategy: byte-aligning chunks (which adds minor overhead).
    final_compressed_bytes = np.concatenate(compressed_chunks)
    
    # Re-calculate final padding based on total bits before padding was added.
    final_padding = (8 - (total_bits % 8)) % 8
    
    header = str(final_padding) + '|' + str(codes_dict_master) + '|'
    
    # Assemble final result
    result_bytes = header.encode('utf-8') + final_compressed_bytes.tobytes()
    
    return result_bytes, N, end_time - start_time

5. Evaluation Framework and Metrics

==============================================================================

In [ ]:
def run_trial(name, runner_func, data, trials=3):
"""Runs multiple trials and calculates average metrics."""
results = []

for _ in range(trials):
    # Time the operation
    start_time = time.time()
    compressed_data, original_size, elapsed_time = runner_func(data)
    
    # Metrics Calculation
    compression_ratio = original_size / len(compressed_data)
    throughput_mbps = (original_size / (1024 * 1024)) / elapsed_time
    
    results.append({
        'throughput_mbps': throughput_mbps,
        'compression_ratio': compression_ratio,
        'elapsed_time': elapsed_time
    })
    
avg_results = {
    'name': name,
    'avg_throughput': np.mean([r['throughput_mbps'] for r in results]),
    'avg_ratio': np.mean([r['compression_ratio'] for r in results]),
}
return avg_results


Wrapper function for sequential test case

In [ ]:


def run_sequential_wrapper(data):
compressed, original_size = cpu_huffman_encode(data)
# The elapsed time is already included in the compressed data from cpu_huffman_encode
# We will redefine the sequential wrapper for clean metrics collection.

start = time.time()
compressed, original_size = cpu_huffman_encode(data)
end = time.time()

# Return structure matching the run_multi_gpu_encoding output
return compressed, original_size, end - start

Wrapper function for single-GPU test case

In [ ]:
def run_single_gpu_wrapper(data_cpu):
# This uses the ParallelHuffman class initialized with one GPU (ID 0)
huffman_runner = ParallelHuffman(gpu_ids=[0])
return huffman_runner.run_multi_gpu_encoding(data_cpu)

--- Main Benchmarking Function ---

def benchmark_all_test_cases(file_size_mb=10, gpu_counts=[1, 2, 4]):

print(f"--- Running Benchmarks for File Size: {file_size_mb} MB ---")
data = generate_test_data(file_size_mb, redundancy_level='high')
all_results = []

# Test Case 1: Standard Sequential (One CPU)
print("1. Testing Standard Sequential (CPU)...")
result_seq = run_trial("Sequential CPU", run_sequential_wrapper, data)
all_results.append(result_seq)
print(f"   -> Throughput: {result_seq['avg_throughput']:.2f} MB/s, Ratio: {result_seq['avg_ratio']:.3f}")

# Test Case 2: CPU Baseline Parallelization (Not explicitly coded, but the Numba/CuPy approach is faster than OpenMP/pthreads and serves as the primary acceleration target. We will focus on the GPU comparison.)
# Note: For simplicity in this environment, we omit a separate OpenMP/pthreads implementation and focus on GPU vs. Sequential.

# Test Case 3: Parallelized across GPU's (Multi-GPU, varying counts)
for gpus in gpu_counts:
    # Note: Set CUDA_VISIBLE_DEVICES environment variable before running this in a true terminal.
    # In a Jupyter notebook, this will typically use the first N visible devices.
    if gpus == 1:
        name = "Single GPU (ID 0)"
        print(f"2. Testing {name}...")
        runner = run_single_gpu_wrapper
    else:
        name = f"Multi-GPU ({gpus} GPUs)"
        print(f"3. Testing {name}...")
        
        # Ensure enough devices are visible/available on the server
        try:
            # Create the runner with the requested number of devices
            runner = lambda d: ParallelHuffman(gpu_ids=list(range(gpus))).run_multi_gpu_encoding(d)
        except cp.cuda.runtime.CudaAPIError:
            print(f"   -> WARNING: Could not initialize {gpus} GPUs. Skipping.")
            continue

    result_gpu = run_trial(name, runner, data)
    all_results.append(result_gpu)
    print(f"   -> Throughput: {result_gpu['avg_throughput']:.2f} MB/s, Ratio: {result_gpu['avg_ratio']:.3f}")
    
return all_results

6. Plotting Results

==============================================================================

In [ ]:
def plot_results(results_list, filesize_mb):
names = [r['name'] for r in results_list]
throughputs = [r['avg_throughput'] for r in results_list]
ratios = [r['avg_ratio'] for r in results_list]

# Plot 1: Throughput (Scalability)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)

# Calculate Speedup relative to the Sequential CPU baseline
seq_throughput = throughputs[0]
speedups = [t / seq_throughput for t in throughputs]

plt.bar(names, throughputs, color=['gray'] + ['darkblue'] * (len(names)-1))
plt.title(f'Throughput Comparison ({filesize_mb} MB File)', fontsize=14)
plt.ylabel('Throughput (MB/s)')

# Add Speedup text on top of the bars
for i, speedup in enumerate(speedups):
    if i == 0:
        plt.text(i, throughputs[i] + 0.05 * max(throughputs), f'{1.00:.2f}x', ha='center', color='black')
    else:
        plt.text(i, throughputs[i] + 0.05 * max(throughputs), f'{speedup:.2f}x', ha='center', color='darkred', weight='bold')

# Plot 2: Compression Ratio (Verification)
plt.subplot(1, 2, 2)
plt.bar(names, ratios, color=['green'] * len(names))
plt.title(f'Compression Ratio Verification ({filesize_mb} MB File)', fontsize=14)
plt.ylabel('Compression Ratio (Original Size / Compressed Size)')
plt.ylim(min(ratios) * 0.9, max(ratios) * 1.1)

# Add ratio text
for i, ratio in enumerate(ratios):
    plt.text(i, ratios[i] + 0.005, f'{ratio:.3f}', ha='center')

plt.tight_layout()
plt.show()

7. Execution Block

==============================================================================

In [ ]:
if name == 'main':
# --- IMPORTANT ---
# 1. Before running Multi-GPU tests, set the environment variable:
#    E.g., In your Jupyter Terminal: export CUDA_VISIBLE_DEVICES=0,1,2,3
# 2. You can vary these parameters for your trials:
FILE_SIZE_TO_TEST = 50 # Start with 50MB, then increase to 500MB+ for better GPU utilization
GPU_CONFIGS = [1, 2]  # Test 1 GPU and 2 GPUs (if available)

# Run the full benchmark
final_results = benchmark_all_test_cases(
    file_size_mb=FILE_SIZE_TO_TEST,
    gpu_counts=GPU_CONFIGS
)

# Generate the performance plots
plot_results(final_results, FILE_SIZE_TO_TEST)

print("\n--- Project Conclusion Focus ---")
print("Your final report should analyze:")
print("1. **Throughput & Scalability:** How much speedup was achieved over the Sequential CPU baseline, and did performance scale linearly with GPU count?")
print("2. **Bottlenecks:** Which stage (Frequency Count, Code Assignment, or Bitstream Encoding/Decoding) was the slowest, and why (Hint: Code Assignment is inherently sequential; Bitstream Encoding is challenging due to complex bit-packing/unaligned memory access).")
print("3. **Compression Efficiency:** Verify that the Compression Ratio is consistent across all implementations.")
